In [106]:
import sys
#!{sys.executable} -m pip install git+https://github.com/Corollarium/geograpy
!{sys.executable} -m pip install geopy

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import json
import nltk
import certifi
import ssl
import time
import pandas as pd
import tika.tika
from tika import parser

In [2]:
df = pd.read_parquet('../documents_processed.parquet')

In [3]:
df.head()

,abstract,body_text,sha,doi_x,doi_y,source_x,journal,has_full_text
0,Bordetella bronchiseptica isolate KM22 has bee...,20-kb insert library preparation protocol (htt...,8f8eb4f004c2002face0723f2f58cc411954d36e,http://dx.doi.org/10.1128/MRA.01207-19,http://dx.doi.org/10.1128/MRA.01207-19,PMC,Microbiol Resour Announc,True
1,,I nfectious diseases have been an ever-present...,63f7049d200896290b38b38711113054f7ea1b50,http://dx.doi.org/10.3201/eid1104.041167,http://dx.doi.org/10.3201/eid1104.041167,PMC,Emerg Infect Dis,True
2,The influenza A nucleoprotein (NP) is an attra...,The transmission of a pathogenic avian H5N1 vi...,4df45b8404d9de0b376a8ae3c282a517df36fe51,http://dx.doi.org/10.1051/vetres/2009071,http://dx.doi.org/10.1051/vetres/2009071,PMC,Vet Res,True
3,The outbreak of severe acute respiratory syndr...,acute lung injury; inflammatory response; neut...,e0737ee93afe7b0bf06b1e3f9adf21d541dd10f0,http://dx.doi.org/10.1038/labinvest.2012.92,http://dx.doi.org/10.1038/labinvest.2012.92,PMC,Lab Invest,True
4,The prevalence of feline herpesvirus-1 (FHV-1)...,Feline herpesvirus type 1 (FHV-1) is the most ...,3c3572ba243d61e7631725669c8f88347fdbd5bc,http://dx.doi.org/10.4142/jvs.2008.9.2.207,http://dx.doi.org/10.4142/jvs.2008.9.2.207,PMC,J Vet Sci,True


In [ ]:
os.mkdir("raw_body_text")

In [ ]:
os.mkdir("geotopic_output")

In [ ]:
for p in df.index:
    f = open("raw_body_text/" + df['sha'][p] + ".geot", "a")
    f.write(df['body_text'][p])
    f.close()

In [ ]:
home = os.getenv('HOME')

In [ ]:
tika.tika.TikaServerClasspath = '/Users/anastasia/geotopicparser-utils/mime:/Users/anastasia/geotopicparser-utils/models/polar:/Users/anastasia/geotopic-mime/tika-config.xml'
parsed = parser.from_file(home + '/geotopicparser-utils/geotopics/polar.geot')
#if 'org.apache.tika.parser.csv.TextAndCSVParser' in parsed["metadata"]['X-Parsed-By']:
print(parsed["metadata"])

In [ ]:
tika.tika.TikaServerClasspath = home + '/geotopicparser-utils/mime:'+home+'/geotopicparser-utils/models/polar'
for file in os.listdir("./raw_body_text"):
    parsed = parser.from_file("./raw_body_text/" + file)
    if 'org.apache.tika.parser.geo.topic.GeoParser' not in parsed["metadata"]['X-Parsed-By']:
        continue
    print("YES")
    json_object = json.dumps(parsed["metadata"], indent = 4) 
    with open("geotopic_output/" + os.path.basename(file).split('.')[0] + ".json", "w") as outfile: 
        outfile.write(json_object) 

In [ ]:
# Another method through the terminal:

! java -classpath tika-app-1.24.jar:$HOME/src/location-ner-model:$HOME/src/geotopic-mime org.apache.tika.cli.TikaCLI -m /Users/anastasia/Desktop/JPL_Remote/covid_19/github_repo/bert_experiments/raw_body_text/0a00a6df208e068e7aa369fb94641434ea0e6070.geot

# ===============================================================

# GeoText

In [6]:
def correct_locations(locations):
    correct_locations = []
    for city in locations:
        correct = True
        tagged = nltk.pos_tag(nltk.word_tokenize(city))
        for tup in tagged:
            if tup[1] != 'NNP' and tup[1] != 'NNPS':
                correct = False
                break
        if correct:
            correct_locations.append(city)
    return correct_locations

In [7]:
def multiple_mentions(data):
    new_geodata = {}
    for paper in data:
        new_geodata[paper] = {}
        for loc in data[paper]:
            seen = set()
            unique = {}
            for place in data[paper][loc]:
                if place not in seen:
                    unique[place] = 1
                    seen.add(place)
                else:
                    unique[place] += 1
            new_geodata[paper][loc] = unique
    return new_geodata

In [8]:
from geopy import geocoders 
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

def get_lat_lon(data):
    ctx = ssl.create_default_context(cafile=certifi.where())
    geopy.geocoders.options.default_ssl_context = ctx
    gn = geocoders.GeoNames(username="anastasija")
    geolocator = Nominatim(user_agent="anastasija", timeout=10, scheme='http')
    geoc = RateLimiter(geolocator.geocode, min_delay_seconds=2)
    for paper in data:
        for loc in data[paper]:
            for place in data[paper][loc]:
                count = data[paper][loc][place]
                data[paper][loc][place] = {}
                try:
                    location = geolocator.geocode(place)
                    data[paper][loc][place]['lat'] = location.latitude
                    data[paper][loc][place]['lon'] = location.longitude
                except:
                    pass
                data[paper][loc][place]['mentioned'] = count
                time.sleep(0.5)
    return data

In [9]:
from geotext import GeoText

geodata = {}
for file in os.listdir("./raw_body_text"):
    with open("./raw_body_text/" + file, 'r') as f:
        text = f.read()
    places = GeoText(text)
    sha = os.path.basename(file).split('.')[0]
    geodata[sha] = {}
    geodata[sha]["countries"] = correct_locations(places.countries)
    geodata[sha]["cities"] = correct_locations(places.cities)

In [10]:
geodata_final = get_lat_lon(multiple_mentions(geodata))

In [11]:
with open('./geodata/geo.json', 'w') as fp:
    json.dump(geodata_final, fp, indent=4)

In [145]:
geodata_final

{'fd31ade73c720f893a5ddb2370df90a84511ec4a': {'countries': {'Brazil': {'lat': -10.3333333,
    'lon': -53.2,
    'mentioned': 14},
   'United States': {'lat': 39.7837304, 'lon': -100.4458825, 'mentioned': 1},
   'Germany': {'lat': 51.0834196, 'lon': 10.4234469, 'mentioned': 1},
   'Italy': {'lat': 42.6384261, 'lon': 12.674297, 'mentioned': 1},
   'Spain': {'lat': 39.3262345, 'lon': -4.8380649, 'mentioned': 1},
   'Taiwan': {'lat': 23.9739374, 'lon': 120.9820179, 'mentioned': 1}},
  'cities': {'Belo Horizonte': {'lat': -19.9227318,
    'lon': -43.9450948,
    'mentioned': 1},
   'Iowa': {'lat': 41.9216734, 'lon': -93.3122705, 'mentioned': 1},
   'São Paulo': {'lat': -23.5506507, 'lon': -46.6333824, 'mentioned': 1}}},
 '0b65341c3090421acaac37ec4c93212277be55bb': {'countries': {}, 'cities': {}}}